In [18]:
import pandas as pd
import numpy as np

In [19]:
path = './pud26/English-all.heads'

f = open(path, "r")
data = []

for x in f:
    data += [[int(y) for y in x.split(' ')]]

print(len(data[0]))
print(data)

30
[[8, 8, 6, 6, 6, 2, 8, 18, 12, 12, 12, 8, 15, 15, 18, 17, 15, 0, 18, 22, 22, 25, 22, 22, 18, 29, 29, 29, 25, 25], [2, 16, 4, 2, 6, 7, 4, 10, 10, 7, 16, 16, 16, 15, 16, 0], [15, 4, 4, 15, 8, 8, 8, 4, 10, 8, 10, 14, 14, 15, 0, 21, 18, 21, 21, 21, 15, 23, 21, 26, 26, 23, 21, 31, 31, 31, 27], [3, 3, 0, 5, 3, 8, 8, 5, 10, 5, 17, 13, 17, 16, 16, 13, 3, 20, 20, 17, 22, 3, 24, 22, 24, 29, 29, 29, 25, 32, 32, 29, 35, 35, 32], [3, 3, 5, 5, 0, 11, 11, 7, 11, 11, 5], [4, 4, 4, 14, 9, 9, 9, 9, 4, 12, 14, 14, 14, 0], [0, 1, 4, 1, 6, 1, 6], [3, 3, 7, 6, 6, 7, 0, 10, 10, 7, 12, 7, 15, 15, 12, 7, 16, 19, 17, 23, 22, 23, 19, 25, 23, 29, 28, 29, 19, 32, 32, 19], [2, 5, 5, 5, 0, 11, 11, 7, 11, 11, 5, 14, 14, 11, 17, 17, 18, 11], [7, 4, 4, 7, 7, 7, 0], [3, 1, 14, 6, 6, 3, 10, 10, 10, 3, 12, 14, 14, 0, 16, 14, 18, 14, 18], [6, 6, 6, 6, 6, 0, 8, 6, 8, 11, 6, 14, 14, 11, 16, 11, 16], [2, 4, 4, 0, 4, 7, 5, 9, 4, 14, 14, 14, 14, 9, 16, 14, 18, 16], [2, 0, 4, 2, 6, 4, 9, 9, 2, 12, 12, 9, 2, 13, 14, 14, 20, 20

In [20]:
def calcular_distancias(grafo):
    n = len(grafo)
    distancias = [[float('inf')] * n for _ in range(n)]

    # Inicializar las distancias conocidas
    for i in range(n):
        distancias[i][i] = 0
        vecino = grafo[i]
        if vecino != 0:
            distancias[i][vecino - 1] = 1  # Peso siempre es 1
            distancias[vecino - 1][i] = 1  # Asegurar bidireccionalidad

    # Calcular las distancias mínimas
    for k in range(n):
        for i in range(n):
            for j in range(n):
                distancias[i][j] = min(distancias[i][j], distancias[i][k] + distancias[k][j])

    return [sum(d) for d in distancias]

# LAL
Primero miramos si el conjunto de arboles es correcto.
Luego procesamos el conjunto de arboles y generamos un fichero CSV con sus métricas

In [11]:
import lal

errlist = lal.io.check_correctness_treebank(path)
for err in errlist:
    print(err)

In [13]:
err = lal.io.process_treebank(path, 'output_file.csv')
if err:
    print(err)

In [16]:
import csv

metrics = []

with open('output_file.csv', 'r') as file:
    csv_reader = csv.DictReader(file, delimiter='\t')
    for row in csv_reader:
        metrics.append(row)

#print(metrics)

[{'n': '30', 'second_moment_degree': '5.8', 'second_moment_degree_out': '3.16667', 'third_moment_degree': '22.3333', 'third_moment_degree_out': '10.9667', 'sum_squared_degrees': '174', 'sum_squared_out_degrees': '95', 'sum_cubed_degrees': '670', 'sum_cubed_out_degrees': '329', 'num_pairs_independent_edges': '348', 'head_initial': '0.310345', 'hubiness': '0.0793651', 'sum_hierarchical_distance': '72', 'mean_hierarchical_distance': '2.48276', 'tree_centre1': '7', 'tree_centre2': '17', 'tree_centroid1': '17', 'tree_centroid2': '31', 'tree_diameter': '7', 'caterpillar_distance': '10', 'num_crossings': '0', 'predicted_num_crossings': '19.4385', 'exp_num_crossings': '116', 'var_num_crossings': '509.511', 'z_score_num_crossings': '-5.13903', 'sum_edge_lengths': '85', 'exp_sum_edge_lengths': '299.667', 'exp_sum_edge_lengths_projective': '109.833', 'exp_sum_edge_lengths_planar': '146.589', 'var_sum_edge_lengths': '1358.49', 'z_score_sum_edge_lengths': '-5.8242', 'min_sum_edge_lengths': '59', 'm

In [35]:
dataframe_dict = {
    'vertex_degree': [],
    'vertex_distance': [],
    'centre': [],
    'centroid': [],
    'class': [],
}

for sentence, m in zip(data, metrics):
    dataframe_dict['class'] += [1 if word == 0 else 0 for word in sentence]
    dataframe_dict['centre'] += [1 if int(m['tree_centre1']) == i or int(m['tree_centre2']) == i else 0 for i in range(len(sentence))]
    dataframe_dict['centroid'] += [1 if int(m['tree_centroid1']) == i or int(m['tree_centroid2']) == i else 0 for i in range(len(sentence))]
    dataframe_dict['vertex_degree'] += [sentence.count(index + 1) for index in range(len(sentence))]
    dataframe_dict['vertex_distance'] += calcular_distancias(sentence)

#print(dataframe_dict['centre'])

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 

In [34]:
dataframe = pd.DataFrame(dataframe_dict)
print(data[1])
dataframe.head(30)

[2, 16, 4, 2, 6, 7, 4, 10, 10, 7, 16, 16, 16, 15, 16, 0]


,vertex_degree,vertex_distance,centre,centroid,class
0,0,106,0,0,0
1,1,98,0,0,0
2,0,148,0,0,0
3,0,148,0,0,0
4,0,148,0,0,0
5,3,120,0,0,0
6,0,106,0,0,0
7,4,78,1,0,0
8,0,128,0,0,0
9,0,128,0,0,0
